In [7]:
from nets import *
from utils import *
from loss import *
from train import batch_generator, read_data_from_batch
import parameter

ModuleNotFoundError: No module named 'nets'

In [4]:
x_path = '../../DDSM/path_file/img_train.txt'
y_path = '../../DDSM/path_file/label_train.txt'
yolo_weight = 'weights/yolov3.weights'

learning_rate = 0.0001
n_classes = 1
input_shape = (416,416)
batch_size = 8

dim_0 = input_shape[0] / 32
dim_1 = dim_0 * 2
dim_2 = dim_1 * 2
dim = (dim_0**2 + dim_1**2 + dim_2**2) * 3

with open(y_path, 'r') as file:
    m = len(file.read().splitlines())

FileNotFoundError: [Errno 2] No such file or directory: '../../DDSM/path_file/label_train.txt'

In [4]:
tf_x = tf.placeholder(tf.float32, [batch_size, input_shape[0], input_shape[1], 1])
tf_y = tf.placeholder(tf.float32, [batch_size, dim, 5+n_classes])

In [5]:
with tf.device('/device:GPU:0'):
    with tf.variable_scope('detector'):
        detections, raw_output = yolo_v3(input_tensor=tf_x, n_classes=n_classes)
        #load_ops = load_weight(var_list=tf.global_variables(scope='detector'), weight_file=yolo_weight, for_training=False)

    boxes = get_boxes(detections, n_classes, input_shape)

    xy_loss, wh_loss, confidence_loss, class_loss = get_loss(raw_output=raw_output, y_true=tf_y, input_shape=input_shape)

    loss = xy_loss + wh_loss + confidence_loss + class_loss

    train_op = tf.train.AdamOptimizer(learning_rate).minimize(loss)

In [6]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [7]:
%%time
for step in range(120):
    
    batches = batch_generator(x_path, y_path, batch_size)
    n_batch = len(batches)
    
    loss_0 = loss_1 = loss_2 = loss_3 = 0
    
    i = 0
    for batch in batches:
        b_x, b_y = read_data_from_batch(batch, input_shape, parameter._ANCHORS, n_classes)
        i += 1
        
    
        _, xy_loss_, wh_loss_, confidence_loss_, class_loss_ = sess.run([train_op, xy_loss, wh_loss, confidence_loss, class_loss], 
                                                                         feed_dict={tf_x:b_x, tf_y:b_y})
        
        print(i)
        loss_0 += xy_loss_
        loss_1 += wh_loss_
        loss_2 += confidence_loss_
        loss_3 += class_loss_
    
    loss_0 /= n_batch
    loss_1 /= n_batch
    loss_2 /= n_batch
    loss_3 /= n_batch
        
    print('Step:', step, '| xy_loss: %.4f' % loss_0 , '| wh_loss: %.4f' % loss_1, '| confidence_loss: %.4f' % loss_2, '| class_loss: %.4f' % loss_3)

1
2


KeyboardInterrupt: 

In [ ]:
saver = tf.train.Saver()

In [ ]:
saver.save(sess, save_path='check/', write_meta_graph=False)